# Realizo la clasificacion usando Gradient Boosting

## Tabla de resultados

In [1]:
#PCA(n)   test_size(n)         Jupyter            Kaggle
#n=13        0.1            0.8281333186898243
#n=12        0.1            0.8292232141456297
#n=11        0.1            0.8274157867486703
#n=10        0.1            0.8258374256711173
#n=9         0.1            0.8272337888510598
#n=8         0.1            0.8267066225269467
#n=7         0.1            0.8241973526627652
#n=6         0.1            0.8269911249875792
#n=5         0.1            0.8233825459832331
#n=4         0.1            0.8257286453185225   0.86161
#n=3         0.1            0.8224233961435273   0.86331
#n=2         0.1            0.8224233961435273   0.86331
#n=1         0.1            0.7662948261344797   0.84722
#Sin PCA     0.1            0.8295024867816182   0.86086

## Importo librerias necesarias

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
#PCA
from sklearn.decomposition import PCA

## Leo todos los datos

In [3]:
df = pd.read_csv('../../data/final_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8', dtype={
    'titulo_analista': np.int8,
    'titulo_tecnico': np.int8,
    'titulo_administrativo': np.int8,    
    'titulo_vendedor': np.int8,          
    'titulo_comercial': np.int8,        
    'titulo_ventas': np.int8,            
    'titulo_sr': np.int8,                
    'titulo_importante': np.int8,        
    'titulo_jefe': np.int8,              
    'titulo_empresa': np.int8,           
    'titulo_asistente': np.int8,         
    'titulo_supervisor': np.int8,         
    'titulo_jr': np.int8,                
    'titulo_contable': np.int8,          
    'desc_experiencia': np.int8,         
    'desc_empresa': np.int8,             
    'desc_trabajo': np.int8,             
    'desc_excluyente': np.int8,          
    'desc_manejo': np.int8,          
    'desc_importante': np.int8,          
    'desc_equipo': np.int8,              
    'desc_tareas': np.int8,              
    'desc_cliente': np.int8,            
    'desc_clientes': np.int8,            
    'desc_conocimientos': np.int8 
})
    
df.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo,idpostulante_int
0,1112448580,1,0,1,0,0,0,0,0,0,...,5,136,439,rmRqLJA,5.414380e+06,10.0,2,25,1,419937
1,1112288191,0,0,0,0,0,0,0,0,0,...,3,118,1546,96XeeQ8,1.196588e+07,-1.0,-1,-1,1,112588
2,1112339031,1,0,0,0,0,0,0,0,0,...,0,24,160,N5lJLJ,1.014115e+07,11.0,2,30,1,205868
3,1112198634,0,0,0,0,0,0,0,0,0,...,0,116,202,RzrrPbY,1.057532e+07,12.0,2,27,1,250580
4,1112453169,1,0,0,0,1,0,0,0,0,...,0,182,1131,lDLQ6W8,5.387287e+06,6.0,1,33,1,378841


In [4]:
cols = df.head(0)
cols.drop(['se_postulo', 'idaviso', 'idpostulante'],inplace=True, axis=1)
cols = cols.columns

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
mintimestamp = df['timestamp'].min()
df['timestamp'] = df['timestamp']-mintimestamp
mintimestamp = df['idpostulante_int'].min()
df['idpostulante_int'] = df['idpostulante_int']-mintimestamp

In [6]:
X = np.array(df.loc[:,cols])
y = np.array(df['se_postulo']) 

## Aplico PCA

In [7]:
#pca = PCA(n_components=4)
#pca.fit(X)
#X = pca.transform(X)

## Realizo el Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=7506)

## Defino el clasificador

In [9]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

## Entreno el clasificador

In [10]:
clf.fit(X_train, y_train);

## Reviso el score

In [11]:
clf.score(X_test, y_test)

0.8298469534039362

## Predigo con todos los datos

In [12]:
X_full_data = X
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data

## Armo .csv para subir a Kaggle

In [13]:
top100_full = pd.read_csv("../../test_final_100k.csv", encoding='utf-8')
top100_full = pd.merge(top100_full, df, how='left', on=['idaviso','idpostulante'])
top100_full = top100_full[['id', 'se_postulo', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full.loc[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1), 'sepostulo'] = 1
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']
top100_full.drop('se_postulo', axis=1, inplace=True)
top100_full.to_csv('../../data/final_data/top100k.csv', encoding='utf-8', index=False)